In [1]:
# Import statements go here
import numpy as np
import emoji
import regex
import pandas as pd
import collections

from ipynb.fs.full.WordProcessing import get_processed_data, convert_to_dict, get_words
from extract_tickers import make_dict_title, get_split_text_from_dict, cross_check_txt_ticker

In [19]:
# Preprocessing functions go here

def extract_emojis(input_file):
    emoji_list = []
    emoji_mat = {}
    posts = input_file.readlines()
    for line in posts:
        post = line[0]
        data = regex.findall(r'\X', post)
        for word in data:
            if any(char in emoji.EMOJI_DATA for char in word):
                if word not in emoji_list:
                    emoji_list.append(word)
    
    for key in emoji_list:
        emoji_mat[key] = np.zeros(len(posts)-1)

    i = 0
    for line in posts:
        post = line[0]
        data = regex.findall(r'\X', post)
        for word in data:
            for char in word:
                if emoji_mat.get(char) is not None:
                    emoji_mat[char][i] += 1
        i += 1
    return emoji_mat

In [1]:
# Run preprocessing code
def preprocess(f_in,to_list=False):
    print("[preprocess::extract emojis]")
    f = open(f_in, "r")
    my_dict_emoji = extract_emojis(f)

    # open and process
    print("[preprocess::extract titles and body]")
    data = pd.read_csv(f_in)
    processed=get_processed_data(data,as_list=to_list)
    my_dict_titles = convert_to_dict(processed,"title")
    my_dict_body = convert_to_dict(processed,"body")
    
    print("[preprocess::complete]")
    return my_dict_emoji, my_dict_titles, my_dict_body, processed

In [2]:
def set_labels_simple(dict_data,pd):
    label_markers = ["buy","sell","hold"]
    ii = 0
    dict_labels = collections.OrderedDict()
    for kk, post in dict_data.items():
        #print(ii)
        if isinstance(post, list):
            post = " ".join(post)
        l_post=get_words(post)
        label_tracker={"sell":0, "hold":0, "buy":0}
        for label in label_markers:
            label_counter=l_post.count(label)
            if label in label_tracker:
                label_tracker[label]=label_counter
            #if label_counter>0: 
            #print("label {0} appeared {1}".format(label,label_counter))
            #print(post)
        #print(label_tracker)
        max_key = max(label_tracker, key=label_tracker.get)
        if label_tracker[max_key] == 0:
            #print("No specific mention to buy,hold,or sell --> setting to hold")
            max_key = "hold"
        #print(max_key)
        dict_labels[ii]=max_key
        #if ii==400:
        #    break    
        ii+=1
    return dict_labels

In [3]:
# useage sanity check
#dict_titles[2345]
#dict_body[2345]
#for emoji_val in dict_emojis.keys():
#        print(emoji_val, np.sum(dict_emojis[emoji_val]))

In [4]:
#dict_emojis, dict_titles, dict_body = preprocess('dataset/reddit_wsb.csv')#,to_list=False)

In [24]:
#my_simple_labels = set_labels_simple(dict_titles,1)

In [25]:
#print(my_simple_labels)